In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.layers import Layer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [ ]:
#“This is an example of a path. You can choose the path according to your actual situation.”
#The data used below is a fusion of parameters extracted from the SIRF model and epidemic prevention policy parameters
file_path = "E:/Data/South Korea/South korea.csv"
df = pd.read_csv(file_path,parse_dates=["Date"],index_col=[0])

In [ ]:
scaler = MinMaxScaler()
test_ = df[-215:]
df_for_scaled_test = scaler.fit_transform(test_)
test = df_for_scaled_test

In [ ]:
#The Train data is weighted data
file_path = "E:/Data/weighted data.csv"
train = pd.read_csv(file_path,index_col=[0])

In [ ]:
def timestamp_data(data):
    X = []
    Y = []
    for i in range(SEQ_LEN, len(data) - FUTURE_PERIOD_PREDICT + 1):
        X.append(data[i - SEQ_LEN:i, :18])
        Y.append(data[i + (FUTURE_PERIOD_PREDICT - 1), 18])
    return np.array(X), np.array(Y)

In [ ]:
X_train, y_train = timestamp_data(train)
X_test, y_test = timestamp_data(test)
print('X_test', X_test.shape, 'y_test', y_test.shape)
print("train shape {0}".format(X_train.shape))
print("test shape {0}".format(X_test.shape))

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
def build_model():
    from tensorflow.keras.models import Sequential
    model_lstm_attention = Sequential()
     
    model_lstm_attention.add(Bidirectional(GRU(64,return_sequences=False),input_shape=(SEQ_LEN, 18)))
    model_lstm_attention.add(Dropout(0.2))
    model_lstm_attention.add(Dropout(0.2))
    model_lstm_attention.add(Dense(16, activation="tanh"))
    model_lstm_attention.add(Dropout(0.2))
    model_lstm_attention.add(Dense(1, activation="relu"))
    model_lstm_attention.compile(loss='mean_squared_error', optimizer='adam')
    model_lstm_attention.summary()
    return model_lstm_attention

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min',restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.01, patience=5, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath='E:/model.h5',#This is an example of a model save path. You can set it according to your actual situation.
        monitor='val_loss',
        verbose = 1,
        save_weights_only=True,
        mode='min'
    )

In [ ]:
model_lstm_attention = KerasRegressor(build_fn=build_model,verbose=1)
my_model = model_lstm_attention
my_model.fit(X_train, y_train,
                         batch_size=128,
                         epochs=1000,
                         validation_data=(X_test, y_test),callbacks=[earlyStopping,reduce_lr])
predicted_LSTM_Att = my_model.predict(X_test)

In [ ]:
prediction_copies_array = np.repeat(predicted_LSTM_Att,19, axis=-1)
pred=scaler.inverse_transform(np.reshape(prediction_copies_array,(len(predicted_LSTM_Att),19)))[:,0]
original_copies_array = np.repeat(y_test,19, axis=-1)
original=scaler.inverse_transform(np.reshape(original_copies_array,(len(y_test),19)))[:,18]

In [ ]:
def relative_error(prediction, actual):
        return abs(prediction - actual) / actual

In [ ]:
relative_error(pred,original)
mae = mean_absolute_error(pred,original)
r2 = r2_score(y_true=original,y_pred=pred)
mape = mean_absolute_percentage_error(pred,original)